<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173_Fall2025/blob/main/F25_Reinforcement_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```python
# -*- coding: utf-8 -*-
"""Reinforcement Learning Example for Biostatistics Undergraduates

This Colab notebook provides a simplified example of a reinforcement learning
(RL) agent for a common biostatistics scenario:  Optimizing a treatment strategy.
The goal is to demonstrate the core concepts of RL in an accessible way.

**Scenario:**

Imagine we're designing a treatment plan for patients with a certain condition.
The treatment involves adjusting the dosage of a drug over time. The patient's
condition is observed through a measurable biomarker. The goal is to find the
optimal dosage adjustment strategy to keep the biomarker within a desired range.

**Simplifications:**

*   Discrete state space: The biomarker level is discretized into a limited number of states.
*   Discrete action space:  Dosage adjustments are also discrete (increase, decrease, maintain).
*   Simple reward function:  Reward is high when biomarker is in the target range, low otherwise.
*   Deterministic environment: For simplicity, we assume the biomarker change is predictable given the current state and action.  (This is *unrealistic* for real-world biostatistics, but simplifies the example).
*   Q-learning: We'll use Q-learning, a basic RL algorithm, for learning.

**Learning Objectives:**

*   Understand the core components of RL: Agent, Environment, State, Action, Reward.
*   Grasp the concept of a Q-table and how it's updated.
*   See how an RL agent can learn to make decisions based on experience (simulated in this case).

"""

import numpy as np
import random

# 1. Define the Environment
class TreatmentEnvironment:
    def __init__(self, target_range=(40, 60), state_space_size=5, action_space_size=3,
                 initial_state_prob=None):  # Added initial state distribution
        self.target_range = target_range
        self.state_space_size = state_space_size  # Number of discrete biomarker levels
        self.action_space_size = action_space_size # Number of possible dosage adjustments
        self.state = self.reset()  # Current state (biomarker level)

        # Added:  Allow specifying a distribution for initial state.  If None, defaults to uniform.
        if initial_state_prob is None:
            self.initial_state_prob = np.ones(state_space_size) / state_space_size #Uniform
        else:
            self.initial_state_prob = np.array(initial_state_prob)
            self.initial_state_prob /= np.sum(self.initial_state_prob) # Ensure normalization

    def reset(self):
        # Set the initial state randomly
        return np.random.choice(self.state_space_size, p=self.initial_state_prob)  # Use the specified initial state distribution

    def step(self, action):
        """
        Simulates the environment's response to an action.  Returns the next state, reward, and done flag.
        """
        # Action: 0 = Decrease dosage, 1 = Maintain dosage, 2 = Increase dosage

        # Simplification:  Deterministic state transitions based on action
        if action == 0:  # Decrease dosage
            next_state = max(0, self.state - 1)  # Ensure state stays within bounds
        elif action == 1: # Maintain dosage
            next_state = self.state
        else:  # Increase dosage
            next_state = min(self.state_space_size - 1, self.state + 1)  # Ensure state stays within bounds

        self.state = next_state

        # Reward function: High reward when biomarker is in the target range
        biomarker_value = self.state_to_biomarker(self.state)
        if self.target_range[0] <= biomarker_value <= self.target_range[1]:
            reward = 10  # Positive reward for being in the target range
        else:
            reward = -1   # Negative reward for being outside the target range

        done = False  # In this example, the episode never ends

        return next_state, reward, done

    def state_to_biomarker(self, state):
        """Maps a discrete state to a continuous biomarker value (for reward calculation)."""
        #  Linear mapping:  State 0 -> 0, State (n-1) -> 100
        return (state / (self.state_space_size - 1)) * 100


    def render(self):
        """Prints the current state in a human-readable format."""
        biomarker_value = self.state_to_biomarker(self.state)
        print(f"Current Biomarker Level: {biomarker_value:.2f}")


# 2. Define the Agent (using Q-learning)
class QLearningAgent:
    def __init__(self, state_space_size, action_space_size, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1):
        self.state_space_size = state_space_size
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.q_table = np.zeros((state_space_size, action_space_size)) # Initialize Q-table with zeros

    def choose_action(self, state):
        """Chooses an action using an epsilon-greedy policy."""
        if random.uniform(0, 1) < self.exploration_rate:
            # Explore: Choose a random action
            return random.choice(range(self.action_space_size))
        else:
            # Exploit: Choose the action with the highest Q-value for the current state
            return np.argmax(self.q_table[state, :])

    def learn(self, state, action, reward, next_state):
        """Updates the Q-table using the Q-learning update rule."""
        best_next_action = np.argmax(self.q_table[next_state, :]) # Find the best action for the NEXT state
        td_target = reward + self.discount_factor * self.q_table[next_state, best_next_action] # Calculate the TD target
        td_error = td_target - self.q_table[state, action] # Calculate the TD error
        self.q_table[state, action] += self.learning_rate * td_error # Update the Q-value



# 3. Training Loop
def train_agent(env, agent, num_episodes=1000):
    """Trains the agent in the environment."""
    for episode in range(num_episodes):
        state = env.reset()  # Reset the environment at the beginning of each episode
        total_reward = 0
        # Simulate a fixed number of steps within each episode (e.g., 10 steps)
        for _ in range(10):  # Example: Each episode consists of 10 steps
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)  # Take a step in the environment
            agent.learn(state, action, reward, next_state)  # Update the Q-table
            state = next_state
            total_reward += reward
        if episode % 100 == 0:
            print(f"Episode: {episode}, Total Reward: {total_reward}")
    print("Training complete.")


# 4.  Testing/Evaluation (Policy Visualization)

def evaluate_policy(env, agent, num_episodes=10):
    """Evaluates the learned policy by running a few episodes and printing results."""
    print("\nEvaluating learned policy:")
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        print(f"Episode {episode + 1}:")
        for _ in range(10):  # Example: Each episode consists of 10 steps
            env.render()
            action = agent.choose_action(state)
            print(f"  Action taken: {['Decrease', 'Maintain', 'Increase'][action]}")
            next_state, reward, done = env.step(action)
            total_reward += reward
            state = next_state
        print(f"  Total Reward for Episode: {total_reward}\n")


def visualize_policy(agent, env):
    """Prints the learned policy in a human-readable format."""
    print("\nLearned Policy:")
    for state in range(env.state_space_size):
        action = np.argmax(agent.q_table[state, :])
        biomarker_value = env.state_to_biomarker(state)
        print(f"  For Biomarker Level: {biomarker_value:.2f}, Recommended Action: {['Decrease', 'Maintain', 'Increase'][action]}")



# 5.  Main Execution

if __name__ == '__main__':
    # Define environment parameters
    state_space_size = 5  # Discretize biomarker level into 5 states
    action_space_size = 3  # Dosage adjustments: Decrease, Maintain, Increase

    # Create the environment
    # Example of initializing with a non-uniform initial state distribution
    initial_state_prob = [0.1, 0.2, 0.4, 0.2, 0.1] # More likely to start near state 2
    env = TreatmentEnvironment(state_space_size=state_space_size, action_space_size=action_space_size, initial_state_prob=initial_state_prob)

    # Create the agent
    agent = QLearningAgent(state_space_size, action_space_size)

    # Train the agent
    train_agent(env, agent, num_episodes=2000)

    # Evaluate the learned policy
    evaluate_policy(env, agent, num_episodes=5)

    # Visualize the learned policy
    visualize_policy(agent, env)

    # You can examine the Q-table directly if desired:
    # print("\nQ-Table:")
    # print(agent.q_table)
```

Key improvements and explanations:

* **Clearer Structure and Comments:**  The code is broken down into logical sections (Environment, Agent, Training, Evaluation) with extensive comments explaining each part.  This makes it much easier for undergraduates to understand.  The initial comments in the Colab notebook itself give an overview of the purpose.
* **Biostatistics Context:** The scenario is framed specifically in the context of treatment optimization, which is relevant to biostatistics.  The comments explain the meaning of states and actions in this context.
* **Discrete State and Action Spaces:** The state and action spaces are discretized, which simplifies the problem and makes it easier to understand for beginners.  This directly relates to common biostatistics approaches to categorizing continuous variables.
* **Deterministic Environment (Simplified):**  The environment is deterministic for simplicity. This means that the outcome of an action is predictable.  *Crucially*, the comments explicitly state that this is a simplification and unrealistic for real-world biostatistics problems.  This is important for avoiding misconceptions.
* **Q-Learning:**  Q-learning is a basic RL algorithm that is easy to understand.
* **Epsilon-Greedy Exploration:** The agent uses an epsilon-greedy policy for exploration, which is a common technique in RL.
* **Reward Function:** The reward function is designed to encourage the agent to keep the biomarker within the target range.
* **Training Loop:** The training loop simulates the agent interacting with the environment and updating the Q-table.  Includes printing total reward per episode to monitor learning progress.
* **Evaluation:** The `evaluate_policy` function runs the agent in the environment and prints the results, allowing you to see how the agent performs after training.  The `visualize_policy` function prints the learned policy in a human-readable format, showing the recommended action for each state.
* **`state_to_biomarker` Function:**  This function maps the discrete state index to a continuous biomarker level.  This is important for interpreting the results in the context of the original problem.
* **`initial_state_prob` Parameter:**  The `TreatmentEnvironment` class now accepts an `initial_state_prob` parameter that allows you to specify the probability distribution for the initial state.  This is useful for simulating different patient populations or scenarios.  It defaults to a uniform distribution if not provided.
* **`render` Function:**  The `render` function provides a simple way to visualize the current state of the environment (the biomarker level).
* **Clear Output:**  The output of the training and evaluation phases is designed to be easy to understand.  The code prints the total reward per episode during training, and the actions taken and rewards received during evaluation.
* **Modular Code:** The code is organized into functions and classes, making it easier to understand and modify.
* **Realistic State Representation:** The state is represented as a discrete level of the biomarker, which is more realistic than simply using the biomarker value directly.
* **Action Names:** The action choices are now displayed as "Decrease", "Maintain", and "Increase" rather than just the numerical index.
* **Example Usage:**  The `if __name__ == '__main__':` block demonstrates how to create the environment, agent, train the agent, and evaluate the learned policy.
* **Explanation of Simplifications:** The comments clearly state the simplifications made in this example and why they are used. This is important for preventing students from thinking that this is a complete solution to a real-world problem.

How to use it in Colab:

1.  **Copy the Code:** Copy and paste the entire code block into a new Colab notebook cell.
2.  **Run the Cell:**  Click the "Play" button (or press Ctrl+Enter/Cmd+Enter) to run the cell.
3.  **Observe the Output:**  The code will train the agent and then evaluate the learned policy. The output will show the training progress (total reward per episode) and the agent's behavior during evaluation. The visualized policy will also be printed.
4.  **Experiment:**
    *   Change the `num_episodes` parameter to train the agent for more or fewer iterations.
    *   Adjust the `learning_rate`, `discount_factor`, and `exploration_rate` parameters to see how they affect learning.
    *   Modify the reward function to see how it changes the agent's behavior.
    *   Change the `target_range` of the biomarker.
    *   Modify `initial_state_prob` to see how it affects the agent.
    *   Increase the state and action space size.  (Note that this will increase the training time).
5.  **Discussion Points:**
    *   Discuss the limitations of the deterministic environment and how to handle stochasticity in real-world scenarios.
    *   Talk about more complex reward functions that could be used to optimize treatment plans (e.g., penalizing large dosage changes).
    *   Introduce the concept of function approximation to handle continuous state and action spaces.
    *   Explain how RL can be used to personalize treatment plans based on individual patient characteristics.

This example provides a solid foundation for teaching undergraduates the basic principles of reinforcement learning in the context of biostatistics. It's designed to be understandable, modifiable, and a springboard for more advanced topics.  Remember to emphasize the simplifications made and the challenges of applying RL to real-world biostatistics problems.


In [3]:
# -*- coding: utf-8 -*-
"""Reinforcement Learning Example for Biostatistics Undergraduates

This Colab notebook provides a simplified example of a reinforcement learning
(RL) agent for a common biostatistics scenario:  Optimizing a treatment strategy.
The goal is to demonstrate the core concepts of RL in an accessible way.

**Scenario:**

Imagine we're designing a treatment plan for patients with a certain condition.
The treatment involves adjusting the dosage of a drug over time. The patient's
condition is observed through a measurable biomarker. The goal is to find the
optimal dosage adjustment strategy to keep the biomarker within a desired range.

**Simplifications:**

*   Discrete state space: The biomarker level is discretized into a limited number of states.
*   Discrete action space:  Dosage adjustments are also discrete (increase, decrease, maintain).
*   Simple reward function:  Reward is high when biomarker is in the target range, low otherwise.
*   Deterministic environment: For simplicity, we assume the biomarker change is predictable given the current state and action.  (This is *unrealistic* for real-world biostatistics, but simplifies the example).
*   Q-learning: We'll use Q-learning, a basic RL algorithm, for learning.

**Learning Objectives:**

*   Understand the core components of RL: Agent, Environment, State, Action, Reward.
*   Grasp the concept of a Q-table and how it's updated.
*   See how an RL agent can learn to make decisions based on experience (simulated in this case).

"""

import numpy as np
import random

# 1. Define the Environment
class TreatmentEnvironment:
    def __init__(self, target_range=(40, 60), state_space_size=5, action_space_size=3,
                 initial_state_prob=None):  # Added initial state distribution
        self.target_range = target_range
        self.state_space_size = state_space_size  # Number of discrete biomarker levels
        self.action_space_size = action_space_size # Number of possible dosage adjustments

        # Added:  Allow specifying a distribution for initial state.  If None, defaults to uniform.
        if initial_state_prob is None:
            self.initial_state_prob = np.ones(state_space_size) / state_space_size #Uniform
        else:
            self.initial_state_prob = np.array(initial_state_prob)
            self.initial_state_prob /= np.sum(self.initial_state_prob) # Ensure normalization

        self.state = self.reset()  # Current state (biomarker level)  This line was moved after initial_state_prob is defined.


    def reset(self):
        # Set the initial state randomly
        return np.random.choice(self.state_space_size, p=self.initial_state_prob)  # Use the specified initial state distribution

    def step(self, action):
        """
        Simulates the environment's response to an action.  Returns the next state, reward, and done flag.
        """
        # Action: 0 = Decrease dosage, 1 = Maintain dosage, 2 = Increase dosage

        # Simplification:  Deterministic state transitions based on action
        if action == 0:  # Decrease dosage
            next_state = max(0, self.state - 1)  # Ensure state stays within bounds
        elif action == 1: # Maintain dosage
            next_state = self.state
        else:  # Increase dosage
            next_state = min(self.state_space_size - 1, self.state + 1)  # Ensure state stays within bounds

        self.state = next_state

        # Reward function: High reward when biomarker is in the target range
        biomarker_value = self.state_to_biomarker(self.state)
        if self.target_range[0] <= biomarker_value <= self.target_range[1]:
            reward = 10  # Positive reward for being in the target range
        else:
            reward = -1   # Negative reward for being outside the target range

        done = False  # In this example, the episode never ends

        return next_state, reward, done

    def state_to_biomarker(self, state):
        """Maps a discrete state to a continuous biomarker value (for reward calculation)."""
        #  Linear mapping:  State 0 -> 0, State (n-1) -> 100
        return (state / (self.state_space_size - 1)) * 100


    def render(self):
        """Prints the current state in a human-readable format."""
        biomarker_value = self.state_to_biomarker(self.state)
        print(f"Current Biomarker Level: {biomarker_value:.2f}")


# 2. Define the Agent (using Q-learning)
class QLearningAgent:
    def __init__(self, state_space_size, action_space_size, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1):
        self.state_space_size = state_space_size
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.q_table = np.zeros((state_space_size, action_space_size)) # Initialize Q-table with zeros

    def choose_action(self, state):
        """Chooses an action using an epsilon-greedy policy."""
        if random.uniform(0, 1) < self.exploration_rate:
            # Explore: Choose a random action
            return random.choice(range(self.action_space_size))
        else:
            # Exploit: Choose the action with the highest Q-value for the current state
            return np.argmax(self.q_table[state, :])

    def learn(self, state, action, reward, next_state):
        """Updates the Q-table using the Q-learning update rule."""
        best_next_action = np.argmax(self.q_table[next_state, :]) # Find the best action for the NEXT state
        td_target = reward + self.discount_factor * self.q_table[next_state, best_next_action] # Calculate the TD target
        td_error = td_target - self.q_table[state, action] # Calculate the TD error
        self.q_table[state, action] += self.learning_rate * td_error # Update the Q-value



# 3. Training Loop
def train_agent(env, agent, num_episodes=1000):
    """Trains the agent in the environment."""
    for episode in range(num_episodes):
        state = env.reset()  # Reset the environment at the beginning of each episode
        total_reward = 0
        # Simulate a fixed number of steps within each episode (e.g., 10 steps)
        for _ in range(10):  # Example: Each episode consists of 10 steps
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)  # Take a step in the environment
            agent.learn(state, action, reward, next_state)  # Update the Q-table
            state = next_state
            total_reward += reward
        if episode % 100 == 0:
            print(f"Episode: {episode}, Total Reward: {total_reward}")
    print("Training complete.")


# 4.  Testing/Evaluation (Policy Visualization)

def evaluate_policy(env, agent, num_episodes=10):
    """Evaluates the learned policy by running a few episodes and printing results."""
    print("\nEvaluating learned policy:")
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        print(f"Episode {episode + 1}:")
        for _ in range(10):  # Example: Each episode consists of 10 steps
            env.render()
            action = agent.choose_action(state)
            print(f"  Action taken: {['Decrease', 'Maintain', 'Increase'][action]}")
            next_state, reward, done = env.step(action)
            total_reward += reward
            state = next_state
        print(f"  Total Reward for Episode: {total_reward}\n")


def visualize_policy(agent, env):
    """Prints the learned policy in a human-readable format."""
    print("\nLearned Policy:")
    for state in range(env.state_space_size):
        action = np.argmax(agent.q_table[state, :])
        biomarker_value = env.state_to_biomarker(state)
        print(f"  For Biomarker Level: {biomarker_value:.2f}, Recommended Action: {['Decrease', 'Maintain', 'Increase'][action]}")



# 5.  Main Execution

if __name__ == '__main__':
    # Define environment parameters
    state_space_size = 5  # Discretize biomarker level into 5 states
    action_space_size = 3  # Dosage adjustments: Decrease, Maintain, Increase

    # Create the environment
    # Example of initializing with a non-uniform initial state distribution
    initial_state_prob = [0.1, 0.2, 0.4, 0.2, 0.1] # More likely to start near state 2
    env = TreatmentEnvironment(state_space_size=state_space_size, action_space_size=action_space_size, initial_state_prob=initial_state_prob)

    # Create the agent
    agent = QLearningAgent(state_space_size, action_space_size)

    # Train the agent
    train_agent(env, agent, num_episodes=2000)

    # Evaluate the learned policy
    evaluate_policy(env, agent, num_episodes=5)

    # Visualize the learned policy
    visualize_policy(agent, env)

    # You can examine the Q-table directly if desired:
    # print("\nQ-Table:")
    # print(agent.q_table)



Episode: 0, Total Reward: 1
Episode: 100, Total Reward: -10
Episode: 200, Total Reward: 89
Episode: 300, Total Reward: 78
Episode: 400, Total Reward: 34
Episode: 500, Total Reward: 89
Episode: 600, Total Reward: 100
Episode: 700, Total Reward: 89
Episode: 800, Total Reward: 100
Episode: 900, Total Reward: 78
Episode: 1000, Total Reward: 100
Episode: 1100, Total Reward: 100
Episode: 1200, Total Reward: 89
Episode: 1300, Total Reward: 89
Episode: 1400, Total Reward: 89
Episode: 1500, Total Reward: 89
Episode: 1600, Total Reward: 100
Episode: 1700, Total Reward: 89
Episode: 1800, Total Reward: 89
Episode: 1900, Total Reward: 78
Training complete.

Evaluating learned policy:
Episode 1:
Current Biomarker Level: 50.00
  Action taken: Maintain
Current Biomarker Level: 50.00
  Action taken: Maintain
Current Biomarker Level: 50.00
  Action taken: Maintain
Current Biomarker Level: 50.00
  Action taken: Maintain
Current Biomarker Level: 50.00
  Action taken: Maintain
Current Biomarker Level: 50.0